In [1]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft


In [3]:
FS = 500                 # Sampling frequency
MAX_LEN = 7500           # ECG length
WINDOW_SIZE = 1000       # 2 seconds
OVERLAP = 0.5
MAX_FREQ = 40            # ECG useful frequency
MAT_DIR = r'D:\ECG_model\Data'
CSV_PATH = r'D:\ECG_model\REFERENCE.csv'
OUTPUT_DIR = r'D:\ECG_model\ecg_images'


In [4]:
def create_ecg_windows(signal, window_size=WINDOW_SIZE, overlap=OVERLAP):
    step = int(window_size * (1 - overlap))
    windows = []
    for start in range(0, len(signal) - window_size + 1, step):
        windows.append(signal[start:start + window_size])
    return windows


In [5]:
def stockwell_transform(signal):
    """
    signal: 1D numpy array (normalized)
    return: 2D Stockwell magnitude (freq x time)
    """
    N = len(signal)
    X = fft(signal)
    t = np.arange(N)

    S = np.zeros((N // 2, N), dtype=np.complex64)

    for k in range(1, N // 2):
        sigma = max(k, 1)
        gaussian = np.exp(-2 * (np.pi ** 2) * ((t - N / 2) ** 2) / (sigma ** 2))
        gaussian = np.roll(gaussian, -N // 2)
        S[k, :] = ifft(X * gaussian)

    return np.abs(S)


In [6]:
def save_stockwell_image(window, save_path):
    # Normalize window (VERY IMPORTANT)
    window = (window - np.mean(window)) / (np.std(window) + 1e-8)

    # Stockwell Transform
    S_mag = stockwell_transform(window)

    # Frequency cropping (0–40 Hz)
    freq_bins = int(MAX_FREQ * WINDOW_SIZE / FS)
    S_mag = S_mag[:freq_bins, :]

    # Log scaling
    S_mag = np.log1p(S_mag)

    # Save image
    plt.figure(figsize=(3, 3))
    plt.imshow(S_mag, aspect="auto", origin="lower", cmap="jet")
    plt.axis("off")
    plt.savefig(save_path, dpi=150, bbox_inches="tight", pad_inches=0)
    plt.close()


In [7]:
labels_df = pd.read_csv(CSV_PATH)
label_dict = dict(zip(labels_df["Recording"], labels_df["First_label"]))


In [8]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

for c in range(1, 10):
    os.makedirs(os.path.join(OUTPUT_DIR, f"class_{c}"), exist_ok=True)


In [9]:
for file in os.listdir(MAT_DIR):
    if not file.endswith(".mat"):
        continue

    record = file.replace(".mat", "")
    if record not in label_dict:
        continue

    label = label_dict[record]

    # Load ECG
    mat = sio.loadmat(os.path.join(MAT_DIR, file))
    signal = mat["ECG"][0][0]["data"][0]   # Lead-1

    # Fix ECG length
    if len(signal) > MAX_LEN:
        signal = signal[:MAX_LEN]
    else:
        signal = np.pad(signal, (0, MAX_LEN - len(signal)))

    # Windowing
    windows = create_ecg_windows(signal)

    # Generate images
    for i, window in enumerate(windows):
        save_path = os.path.join(
            OUTPUT_DIR,
            f"class_{label}",
            f"{record}_win{i}.png"
        )
        save_stockwell_image(window, save_path)

    print(f"Processed {record}")


Processed A0001
Processed A0002
Processed A0003
Processed A0004
Processed A0005
Processed A0006
Processed A0007
Processed A0008
Processed A0009
Processed A0010
Processed A0011
Processed A0012
Processed A0013
Processed A0014
Processed A0015
Processed A0016
Processed A0017
Processed A0018
Processed A0019
Processed A0020
Processed A0021
Processed A0022
Processed A0023
Processed A0024
Processed A0025
Processed A0026
Processed A0027
Processed A0028
Processed A0029
Processed A0030
Processed A0031
Processed A0032
Processed A0033
Processed A0034
Processed A0035
Processed A0036
Processed A0037
Processed A0038
Processed A0039
Processed A0040
Processed A0041
Processed A0042
Processed A0043
Processed A0044
Processed A0045
Processed A0046
Processed A0047
Processed A0048
Processed A0049
Processed A0050
Processed A0051
Processed A0052
Processed A0053
Processed A0054
Processed A0055
Processed A0056
Processed A0057
Processed A0058
Processed A0059
Processed A0061
Processed A0062
Processed A0063
Processe

MemoryError: Unable to allocate 2.44 MiB for an array with shape (80, 1000, 4) and data type float64

Error in callback <function _draw_all_if_interactive at 0x0000020874C72610> (for post_execute), with arguments args (),kwargs {}:


MemoryError: Unable to allocate 2.44 MiB for an array with shape (80, 1000, 4) and data type float64

MemoryError: Unable to allocate 2.44 MiB for an array with shape (80, 1000, 4) and data type float64

<Figure size 300x300 with 1 Axes>

In [ ]:
print('I miss you chapri:(..........)')

: 